# **OpenSSL CheatSheet**

### 1. Generating Keys

* **RSA Private Key (2048-bit):**

  ```bash
  openssl genrsa -out private.key 2048
  ```
* **RSA Private Key with Password:**

  ```bash
  openssl genrsa -aes256 -out private_encrypted.key 2048
  ```
* **Extract Public Key from Private:**

  ```bash
  openssl rsa -in private.key -pubout -out public.key
  ```
* **EC Private Key:**

  ```bash
  openssl ecparam -name prime256v1 -genkey -noout -out ec_private.key
  ```

---

### 2. Certificate Signing Request (CSR)

* **Generate CSR from Private Key:**

  ```bash
  openssl req -new -key private.key -out server.csr
  ```

  > Follow prompts for organization details.

---

### 3. Self-Signed Certificates

* **Generate Self-Signed Cert from Private Key & CSR:**

  ```bash
  openssl x509 -req -days 365 -in server.csr -signkey private.key -out server.crt
  ```
* **Generate Self-Signed Cert directly from Private Key (no separate CSR):**

  ```bash
  openssl req -x509 -new -nodes -days 365 -key private.key -out server.crt
  ```

  > Use `-nodes` for no encryption on the private key.

---

### 4. Viewing Certificates & Keys

* **View CSR content:**

  ```bash
  openssl req -in server.csr -text -noout
  ```
* **View Certificate content:**

  ```bash
  openssl x509 -in server.crt -text -noout
  ```
* **View Private Key details:**

  ```bash
  openssl rsa -in private.key -text -noout
  ```
* **Verify Certificate Chain (if applicable):**

  ```bash
  openssl verify -CAfile ca.crt server.crt
  ```

---

### 5. Encrypt/Decrypt Files (Symmetric - AES256)

* **Encrypt a file:**

  ```bash
  openssl enc -aes256 -salt -in cleartext.txt -out encrypted.enc
  ```

  > Prompts for a password.
* **Decrypt a file:**

  ```bash
  openssl enc -d -aes256 -in encrypted.enc -out decrypted.txt
  ```

  > Prompts for the same password.

---

### 6. Hashing & Message Digests

* **Calculate SHA256 hash of a file:**

  ```bash
  openssl dgst -sha256 file.txt
  ```
* **Calculate MD5 hash of a file:**

  ```bash
  openssl dgst -md5 file.txt
  ```

---

### 7. Base64 Encoding/Decoding

* **Encode a file to Base64:**

  ```bash
  openssl base64 -in input.bin -out output.b64
  ```
* **Decode a Base64 file:**

  ```bash
  openssl base64 -d -in input.b64 -out output.bin
  ```

---

### 8. Symmetric vs. Asymmetric Encryption Basics

* **Symmetric:** Uses the same key for encryption and decryption (e.g., AES256, done with `openssl enc`).

* **Asymmetric:** Uses a pair of keys (public for encryption, private for decryption, or vice-versa for signing) (e.g., RSA, done with `openssl pkeyutl`).

* **Encrypt with Public Key (RSA):**

  ```bash
  openssl pkeyutl -encrypt -pubin -inkey public.key -in cleartext.txt -out encrypted.rsa
  ```

  > Note: RSA encrypts small amounts of data efficiently. Usually, a symmetric key is encrypted by RSA.

* **Decrypt with Private Key (RSA):**

  ```bash
  openssl pkeyutl -decrypt -inkey private.key -in encrypted.rsa -out decrypted.txt
  ```

---

### 9. SSL/TLS Basics (Client/Server Simulation)

* **Start a simple SSL/TLS server:**

  ```bash
  openssl s_server -accept 4433 -cert server.crt -key private.key
  ```

* **Connect to an SSL/TLS server (client):**

  ```bash
  openssl s_client -connect <host>:4433
  ```

  > Use `quit` to exit the client connection.

* **Check a website's SSL certificate:**

  ```bash
  openssl s_client -connect www.google.com:443 -showcerts
  ```

  > Includes the certificate chain.


---


### 10. Digital Signing & Verification

* **Sign a file with an RSA private key**

  ```bash
  openssl dgst -sha256 -sign private.key -out file.sig file.txt
  ```

  > `-sha256` specifies the digest algorithm.
  > `file.sig` is the binary signature.


* **Verify the signature with the RSA public key**

  ```bash
  openssl dgst -sha256 -verify public.key -signature file.sig file.txt
  ```

  > Prints `Verified OK` if the signature matches.


* **Sign a file with an EC (Elliptic Curve) private key**

  ```bash
  openssl dgst -sha256 -sign ec_private.key -out file_ec.sig file.txt
  ```


* **Verify the EC signature**

  ```bash
  openssl dgst -sha256 -verify ec_public.pem -signature file_ec.sig file.txt
  ```

  > For EC, you need the public key in PEM format.
  > (Extract with `openssl ec -in ec_private.key -pubout -out ec_public.pem`.)


* **Sign and output the signature in Base64 (optional)**

  ```bash
  openssl dgst -sha256 -sign private.key file.txt | openssl base64 -A > file.sig.b64
  ```
---

# **Boilerplate Python Code**

In [ ]:
# Python code to run OpenSSL command

import subprocess

# Encrypt
command = [ "openssl", "enc", "-aes-256-cbc", "-md", "sha512", "-pbkdf2", "-iter", "1000", "-salt", "-in", "message.txt", "-out", "encrypt_1.enc" ]
# Decrypt
command = [ "openssl", "enc", "-aes-256-cbc", "-md", "sha512", "-pbkdf2", "-iter", "1000", "-d", "-in", "encrypt_1.enc", "-out", "decrypt_1.txt" ]

try:
    # Run the OpenSSL command
    subprocess.run(command, check=True)
    print("File encrypted and saved to 'encrypt.enc'.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred during encryption: {e}")

# **RSA**

```bash
# Private key
openssl genrsa -out private.key 2048

# Public key (PEM format)
openssl rsa -in private.key -pubout -out public.key

echo "Your Full Name" > input.txt

openssl rsautl -sign -inkey private.key -in input.txt -out input.rsa
```

* `rsautl -sign` → uses **private key** for encryption.
* `-in` / `-out` → plaintext & result.


```bash
openssl rsautl -verify -inkey public.key -pubin -in input.rsa -out recovered.txt
cat recovered.txt   # should show your name

# Full details
openssl rsa -in private.key -text -noout > rsa_params.txt
```

Inside `rsa_params.txt` you’ll see:

```
modulus: n
publicExponent: e
privateExponent: d
prime1: p
prime2: q
```

# **RSA Client-Server**

**generate_keys.py**

In [ ]:
"""
Description:
This script uses OpenSSL to generate an RSA private key and then extracts the
corresponding public key from it. These keys (myprivate.key and mypublic.key)
are essential for the asymmetric encryption and decryption process used in
the client and server scripts. The private key will be used by the server
to decrypt messages, and the public key will be used by the client to encrypt them.
"""
import subprocess

# Command to generate an RSA private key.
# 'genrsa': OpenSSL command for generating RSA keys.
# '-out myprivate.key': Specifies the output file name for the private key.
# By default, it generates a 2048-bit key.
private_key_command = [
    'openssl', 'genrsa', '-out', 'myprivate.key'
]

# Command to extract the public key from the generated private key.
# 'rsa': OpenSSL command for RSA key utilities.
# '-in myprivate.key': Specifies the input private key file.
# '-pubout': Tells OpenSSL to output the public key part.
# '-out mypublic.key': Specifies the output file name for the public key.
public_key_command = [
    'openssl', 'rsa', '-in', 'myprivate.key', '-pubout', '-out', 'mypublic.key'
]

print("Generating RSA private key (myprivate.key)...")
# Execute the private key generation command.
# 'check=True' will raise an error if the command fails.
subprocess.run(private_key_command, check=True)
print("Private key generated.")

print("Extracting public key (mypublic.key) from private key...")
# Execute the public key extraction command.
subprocess.run(public_key_command, check=True)
print("Public key generated.")
print("Keys (myprivate.key, mypublic.key) are ready for use.")

**server.py**

In [ ]:
"""
Description:
This script acts as a server that listens for incoming connections.
It is designed to receive an encrypted file from a client, save it locally,
and then decrypt it using its own RSA private key (myprivate.key).
Finally, it sends a confirmation back to the client.
"""
import socket
import subprocess
import os

def start_server():
    # Create a TCP/IP socket.
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Bind the socket to a specific address and port.
    # '127.0.0.1' is the localhost IP, and 65431 is an arbitrary port number.
    server_socket.bind(('127.0.0.1', 65431))

    # Enable the server to accept connections. The argument specifies
    # the maximum number of queued connections.
    server_socket.listen(1)

    print("Server is listening on port 65431...")

    # Accept a new connection. 'conn' is the new socket object used to
    # communicate with the client, and 'addr' is the client's address.
    conn, addr = server_socket.accept()
    print(f"Connected by {addr}")

    received_filename = "received_file.enc"
    decrypted_filename = "decrypt.txt"
    private_key_filename = "myprivate.key"

    # Open a file to store the received encrypted content in binary write mode.
    with open(received_filename, 'wb') as file:
        while True:
            # Receive data in chunks of 1024 bytes.
            data = conn.recv(1024)
            # Check for the custom end-of-file marker sent by the client.
            if data == b'END':
                break
            # If no more data is received and it's not the 'END' marker,
            # it means the connection might have been closed unexpectedly.
            if not data:
                break
            # Write the received chunk to the file.
            file.write(data)

    print(f"File received and saved as '{received_filename}'")

    # Send a confirmation message back to the client.
    conn.sendall(b"File received successfully")

    # --- Decryption Process ---
    # Ensure the private key exists before attempting decryption.
    if not os.path.exists(private_key_filename):
        print(f"Error: Private key '{private_key_filename}' not found. Cannot decrypt.")
        conn.close()
        server_socket.close()
        return

    # Construct the OpenSSL command to decrypt the received file.
    # 'pkeyutl': OpenSSL utility for public key operations.
    # '-decrypt': Specifies decryption operation.
    # '-in received_file.enc': Input encrypted file.
    # '-inkey myprivate.key': Specifies the private key to use for decryption.
    # '-out decrypt.txt': Output file for the decrypted content.
    command = [
        "openssl", "pkeyutl", "-decrypt", "-in", received_filename,
        "-inkey", private_key_filename, "-out", decrypted_filename
    ]

    print(f"Decrypting '{received_filename}' using '{private_key_filename}'...")
    try:
        # Execute the OpenSSL command. 'check=True' will raise an exception
        # if the command returns a non-zero exit code (indicating an error).
        subprocess.run(command, check=True)
        print(f"File decrypted and saved as '{decrypted_filename}'")
    except subprocess.CalledProcessError as e:
        print(f"Error during decryption: {e}")
        print("Please ensure 'myprivate.key' is correct and the file was encrypted with the corresponding public key.")
    except FileNotFoundError:
        print("Error: openssl command not found. Make sure OpenSSL is installed and in your system's PATH.")

    # Close the connection with the client.
    conn.close()
    # Close the server listening socket.
    server_socket.close()

if __name__ == "__main__":
    start_server()

**client.py**

In [ ]:
"""
Description:
This script acts as a client that connects to a server.
It first encrypts a plaintext file (message.txt) using an RSA public key
(mypublic.key) and then sends the encrypted content to the server.
Finally, it waits for a confirmation message from the server.
"""
import socket
import subprocess
import os

def send_file():
    # Create a TCP/IP socket.
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    server_address = ('127.0.0.1', 65431)
    print(f"Attempting to connect to server at {server_address}...")
    try:
        # Connect to the server's IP address and port.
        client_socket.connect(server_address)
        print("Connected to the server.")
    except ConnectionRefusedError:
        print(f"Error: Connection refused. Is the server running at {server_address}?")
        return
    except Exception as e:
        print(f"An unexpected error occurred while connecting: {e}")
        return

    message_filename = "message.txt"
    encrypted_filename = "encrypt.enc"
    public_key_filename = "mypublic.key"

    # --- Encryption Process ---
    # Ensure the public key and message file exist before attempting encryption.
    if not os.path.exists(public_key_filename):
        print(f"Error: Public key '{public_key_filename}' not found. Cannot encrypt.")
        client_socket.close()
        return
    if not os.path.exists(message_filename):
        print(f"Error: Message file '{message_filename}' not found. Please create it.")
        client_socket.close()
        return

    # Construct the OpenSSL command to encrypt the message file.
    # 'pkeyutl': OpenSSL utility for public key operations.
    # '-encrypt': Specifies encryption operation.
    # '-in message.txt': Input plaintext file.
    # '-pubin': Indicates that the input key is a public key.
    # '-inkey mypublic.key': Specifies the public key to use for encryption.
    # '-out encrypt.enc': Output file for the encrypted content.
    command = [
        "openssl", "pkeyutl", "-encrypt", "-in", message_filename,
        "-pubin", "-inkey", public_key_filename, "-out", encrypted_filename
    ]

    print(f"Encrypting '{message_filename}' using '{public_key_filename}'...")
    try:
        # Execute the OpenSSL command.
        subprocess.run(command, check=True)
        print(f"File encrypted and saved as '{encrypted_filename}'")
    except subprocess.CalledProcessError as e:
        print(f"Error during encryption: {e}")
        print("Please ensure 'mypublic.key' is correct and 'message.txt' exists.")
        client_socket.close()
        return
    except FileNotFoundError:
        print("Error: openssl command not found. Make sure OpenSSL is installed and in your system's PATH.")
        client_socket.close()
        return

    # --- File Transfer Process ---
    # Open the encrypted file in binary read mode.
    print(f"Sending encrypted file '{encrypted_filename}' to server...")
    with open(encrypted_filename, 'rb') as file:
        while True:
            # Read the file in chunks of 1024 bytes.
            chunk = file.read(1024)
            if not chunk:
                break # End of file
            # Send the chunk to the server.
            client_socket.sendall(chunk)

    # Send a custom end-of-file marker to indicate that the entire file has been sent.
    client_socket.sendall(b'END')
    print("Encrypted file sent successfully. Waiting for server confirmation...")

    # Receive the server's confirmation message.
    try:
        data = client_socket.recv(1024)
        print(f"Received from server: {data.decode('utf-8')}")
    except Exception as e:
        print(f"Error receiving confirmation from server: {e}")

    # Close the client socket.
    client_socket.close()
    print("Client connection closed.")

if __name__ == "__main__":
    send_file()

# **RSA Client-Server Certificate**

**generate_keys.py**

In [ ]:
"""
Description:
This script generates the necessary RSA private keys and self-signed X.509
certificates for both the server and the client to establish a secure
TLS-enabled communication channel.

For the server:
- server.key: The server's private key.
- server.crt: The server's self-signed certificate, containing its public key.

For the client:
- client.key: The client's private key.
- client.crt: The client's self-signed certificate, containing its public key.

These certificates are 'self-signed' meaning they are signed by their own
private key, rather than by a trusted Certificate Authority (CA).
In a real-world scenario, certificates would typically be issued by a CA.
For mutual authentication in a lab, each party will "trust" the other's
self-signed certificate.
"""
import subprocess
import os

# --- Configuration for Server Certificate ---
SERVER_KEY = "server.key"
SERVER_CSR = "server.csr"
SERVER_CRT = "server.crt"
SERVER_CN = "/CN=localhost" # Common Name for the server

# --- Configuration for Client Certificate ---
CLIENT_KEY = "client.key"
CLIENT_CSR = "client.csr"
CLIENT_CRT = "client.crt"
CLIENT_CN = "/CN=client" # Common Name for the client

def generate_key_and_cert(key_file, csr_file, crt_file, common_name):
    """Generates a private key, CSR, and self-signed certificate."""
    print(f"\n--- Generating for {os.path.basename(key_file).split('.')[0]} ---")

    # 1. Generate Private Key
    print(f"1. Generating private key: {key_file} (2048-bit RSA)...")
    subprocess.run(["openssl", "genrsa", "-out", key_file, "2048"], check=True)

    # 2. Generate Certificate Signing Request (CSR)
    print(f"2. Generating CSR: {csr_file}...")
    # -new: Creates a new CSR
    # -key: Specifies the private key to use
    # -out: Output CSR file
    # -subj: Subject information (Common Name is critical for TLS)
    subprocess.run([
        "openssl", "req", "-new", "-key", key_file, "-out", csr_file, "-subj", common_name
    ], check=True)

    # 3. Generate Self-Signed Certificate from CSR
    print(f"3. Generating self-signed certificate: {crt_file} (valid for 365 days)...")
    # -x509: Output a self-signed certificate instead of a CSR
    # -req: Expects a CSR as input
    # -days: Validity period in days
    # -in: Input CSR file
    # -signkey: Sign the certificate with the specified private key
    # -out: Output certificate file
    subprocess.run([
        "openssl", "x509", "-req", "-days", "365", "-in", csr_file,
        "-signkey", key_file, "-out", crt_file
    ], check=True)
    print(f"Successfully generated {crt_file} and {key_file}")

if __name__ == "__main__":
    # Generate for Server
    generate_key_and_cert(SERVER_KEY, SERVER_CSR, SERVER_CRT, SERVER_CN)

    # Generate for Client
    generate_key_and_cert(CLIENT_KEY, CLIENT_CSR, CLIENT_CRT, CLIENT_CN)

    print("\nAll keys and self-signed certificates generated. Ready for TLS communication.")
    print("Remember: For mutual authentication, the server will trust 'client.crt' and the client will trust 'server.crt'.")

**server.py**

In [ ]:
"""
Description:
This script implements a TLS-enabled server using Python's 'ssl' module for
secure communication. It sets up mutual authentication, meaning both the
server and client must present valid certificates to each other.

The server loads its own private key (server.key) and certificate (server.crt).
It then configures an SSLContext to:
1. Load its certificate and private key.
2. Require client certificates for verification (CERT_REQUIRED).
3. Trust client certificates signed by 'client.crt' (in a lab, we use client.crt itself
   as the trusted CA for client auth).

Once a secure TLS connection is established, the server receives data,
prints it, and sends a confirmation back.
"""
import socket
import ssl
import os

HOST = '127.0.0.1'
PORT = 65431

SERVER_KEY_FILE = "server.key"
SERVER_CRT_FILE = "server.crt"
# For mutual authentication, the server needs to trust the client's certificate.
# In a real CA scenario, this would be the CA's cert. For self-signed, it's the client's cert.
CLIENT_TRUSTED_CA_FILE = "client.crt"

def start_tls_server():
    # --- Check for existence of required files ---
    for f in [SERVER_KEY_FILE, SERVER_CRT_FILE, CLIENT_TRUSTED_CA_FILE]:
        if not os.path.exists(f):
            print(f"Error: Required file '{f}' not found. Please run 'generate_tls_certs.py' first.")
            return

    # 1. Create an SSLContext
    # PROTOCOL_TLS_SERVER: Uses the highest available TLS version for server-side.
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)

    # 2. Load the server's private key and certificate
    context.load_cert_chain(SERVER_CRT_FILE, SERVER_KEY_FILE)

    # 3. Configure client certificate requirements for mutual authentication
    # CERT_REQUIRED: The server will require a client certificate during the handshake.
    context.verify_mode = ssl.CERT_REQUIRED
    # load_verify_locations: Specifies the CA certificate(s) used to verify client certificates.
    # In this self-signed example, the server trusts the client's specific self-signed certificate.
    context.load_verify_locations(CLIENT_TRUSTED_CA_FILE)

    # Create a regular TCP/IP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((HOST, PORT))
    server_socket.listen(1)

    print(f"TLS Server listening on {HOST}:{PORT} with mutual authentication...")
    print(f"Expecting client certs verified by {CLIENT_TRUSTED_CA_FILE}")

    try:
        # Wrap the regular socket with the SSL context
        # This performs the TLS handshake and handles encryption/decryption.
        with context.wrap_socket(server_socket, server_side=True) as tls_socket:
            conn, addr = tls_socket.accept()
            print(f"Connected by {addr} via TLS")

            # Get client certificate details (optional, for verification/logging)
            client_cert = conn.getpeercert()
            if client_cert:
                print(f"Client certificate subject: {client_cert.get('subject')}")
                print(f"Client certificate issuer: {client_cert.get('issuer')}")
            else:
                print("Warning: Client did not present a certificate or it wasn't verified.")

            print("Receiving data...")
            received_data = b""
            while True:
                data = conn.recv(4096) # Read up to 4KB of data
                if data == b'END_OF_FILE': # Custom end marker
                    break
                if not data:
                    break
                received_data += data

            print(f"\nReceived (decrypted by TLS layer):")
            print("-" * 30)
            print(received_data.decode('utf-8'))
            print("-" * 30)

            # Send a confirmation message (also encrypted by TLS layer)
            confirmation_message = "Message received successfully via TLS!"
            conn.sendall(confirmation_message.encode('utf-8'))
            print("Confirmation sent to client.")

            conn.close()

    except ssl.SSLError as e:
        print(f"TLS/SSL Error: {e}")
        if "CERTIFICATE_VERIFY_FAILED" in str(e):
            print("Possible reasons: Client did not present a certificate, or its certificate is not trusted.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        server_socket.close()
        print("Server closed.")

if __name__ == "__main__":
    start_tls_server()

**client.py**

In [ ]:
"""
Description:
This script acts as a TLS-enabled client that connects to a secure server.
It utilizes Python's 'ssl' module to establish a mutually authenticated
TLS connection.

The client loads its own private key (client.key) and certificate (client.crt).
It then configures an SSLContext to:
1. Load its certificate and private key to present to the server.
2. Require the server's certificate for verification (CERT_REQUIRED).
3. Trust server certificates signed by 'server.crt' (in a lab, we use server.crt itself
   as the trusted CA for server auth).

Once the TLS handshake is complete and certificates are verified, the client
sends a plaintext message, which is automatically encrypted by the TLS layer,
and receives a confirmation from the server.
"""
import socket
import ssl
import os

HOST = '127.0.0.1'
PORT = 65431

CLIENT_KEY_FILE = "client.key"
CLIENT_CRT_FILE = "client.crt"
# For mutual authentication, the client needs to trust the server's certificate.
# In a real CA scenario, this would be the CA's cert. For self-signed, it's the server's cert.
SERVER_TRUSTED_CA_FILE = "server.crt"

def send_tls_message():
    # --- Check for existence of required files ---
    for f in [CLIENT_KEY_FILE, CLIENT_CRT_FILE, SERVER_TRUSTED_CA_FILE]:
        if not os.path.exists(f):
            print(f"Error: Required file '{f}' not found. Please run 'generate_tls_certs.py' first.")
            return

    # 1. Create an SSLContext
    # PROTOCOL_TLS_CLIENT: Uses the highest available TLS version for client-side.
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)

    # 2. Load the client's private key and certificate to present to the server
    context.load_cert_chain(CLIENT_CRT_FILE, CLIENT_KEY_FILE)

    # 3. Configure server certificate requirements
    # CERT_REQUIRED: The client will require and verify the server's certificate.
    context.verify_mode = ssl.CERT_REQUIRED
    # load_verify_locations: Specifies the CA certificate(s) used to verify the server's certificate.
    # In this self-signed example, the client trusts the server's specific self-signed certificate.
    context.load_verify_locations(SERVER_TRUSTED_CA_FILE)

    # Create a regular TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    print(f"TLS Client attempting to connect to {HOST}:{PORT} with mutual authentication...")
    print(f"Presenting client cert {CLIENT_CRT_FILE} and expecting server cert verified by {SERVER_TRUSTED_CA_FILE}")

    try:
        # Wrap the regular socket with the SSL context
        # This initiates the TLS handshake and handles certificate exchange/verification.
        with context.wrap_socket(sock, server_hostname=HOST) as tls_sock:
            tls_sock.connect((HOST, PORT))
            print("Successfully established TLS connection.")

            # Get server certificate details (optional, for verification/logging)
            server_cert = tls_sock.getpeercert()
            if server_cert:
                print(f"Server certificate subject: {server_cert.get('subject')}")
                print(f"Server certificate issuer: {server_cert.get('issuer')}")
            else:
                print("Warning: Server did not present a certificate or it wasn't verified.")


            message = "This is a secret message sent securely over TLS!"
            print(f"Sending (will be encrypted by TLS layer): '{message}'")
            # Send the message (TLS layer encrypts it)
            tls_sock.sendall(message.encode('utf-8'))
            tls_sock.sendall(b'END_OF_FILE') # Custom end marker

            # Receive confirmation (TLS layer decrypts it)
            response = tls_sock.recv(4096)
            print(f"Received (decrypted by TLS layer): '{response.decode('utf-8')}'")

    except ConnectionRefusedError:
        print(f"Error: Connection refused. Is the server running at {HOST}:{PORT}?")
    except ssl.SSLError as e:
        print(f"TLS/SSL Error: {e}")
        if "CERTIFICATE_VERIFY_FAILED" in str(e):
            print("Possible reasons: Server presented an untrusted certificate, or its certificate is invalid.")
        if "handshake failure" in str(e):
            print("Possible reasons: Client certificate not accepted by server, or protocol mismatch.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        sock.close()
        print("Client closed.")

if __name__ == "__main__":
    send_tls_message()

# **Encryption Decryption**

### 1. Command Structure

```bash
openssl enc <cipher> -e -in <plaintext> -out <ciphertext> -K <key in hex> -iv <IV in hex>
openssl enc <cipher> -d -in <ciphertext> -out <decrypted> -K <key in hex> -iv <IV in hex>
```

---

### 2. Meaning of Arguments

| Argument      | Description                                                                |
| ------------- | -------------------------------------------------------------------------- |
| `<cipher>`    | Cipher + mode (e.g., `-aes-128-cbc`, `-aes-192-cfb`, `-bf-cbc`).           |
| `-e`          | **Encrypt** the input file.                                                |
| `-d`          | **Decrypt** the input file.                                                |
| `-in <file>`  | Input file (plaintext for `-e`, ciphertext for `-d`).                      |
| `-out <file>` | Output file (ciphertext for `-e`, plaintext for `-d`).                     |
| `-K <key>`    | **Key** in hex, must match the algorithm’s key size.                       |
| `-iv <iv>`    | **Initialization Vector** in hex, must match the block size of the cipher. |

---

### Keys (`-K`)

* A **key** is a secret used by the cipher to encrypt/decrypt.
* Must be provided as **hexadecimal** (0–9, a–f).
* Length depends on algorithm:

  * AES-128 → 16 bytes (32 hex chars)
  * AES-192 → 24 bytes (48 hex chars)
  * AES-256 → 32 bytes (64 hex chars)
  * Blowfish → 16 bytes (32 hex chars)
* Generate random key:

  ```bash
  openssl rand -hex 16   # 16 bytes = 128 bits
  ```

---

### Initialization Vector (`-iv`)

* The **IV** ensures different ciphertexts even with the same key & plaintext.
* Size = cipher’s block size:

  * AES = 16 bytes (32 hex chars)
  * DES/Blowfish = 8 bytes (16 hex chars)
* Must be the **same for encryption & decryption**.

> If you use the wrong key or IV, decryption will fail or produce garbage.

---

### 3. Prepare a Plaintext

```bash
echo "This is a secret message for testing OpenSSL." > plain.txt
```

---

###  4. Examples (Encryption + Decryption)

---

#### AES-128 in CBC Mode

```bash
# Encrypt
openssl enc -aes-128-cbc -e -in plain.txt -out cipher_aes128cbc.bin \
  -K 00112233445566778899aabbccddeeff \
  -iv 0102030405060708090a0b0c0d0e0f10

# Decrypt
openssl enc -aes-128-cbc -d -in cipher_aes128cbc.bin -out decrypted_aes128cbc.txt \
  -K 00112233445566778899aabbccddeeff \
  -iv 0102030405060708090a0b0c0d0e0f10
```

---

#### AES-192 in CFB Mode

```bash
# Encrypt
openssl enc -aes-192-cfb -e -in plain.txt -out cipher_aes192cfb.bin \
  -K 00112233445566778899aabbccddeeff0011223344556677 \
  -iv 1112131415161718191a1b1c1d1e1f20

# Decrypt
openssl enc -aes-192-cfb -d -in cipher_aes192cfb.bin -out decrypted_aes192cfb.txt \
  -K 00112233445566778899aabbccddeeff0011223344556677 \
  -iv 1112131415161718191a1b1c1d1e1f20
```

---

#### Blowfish in CBC Mode

```bash
# Encrypt
openssl enc -bf-cbc -e -in plain.txt -out cipher_bfcbc.bin \
  -K 00112233445566778899aabbccddeeff \
  -iv 0102030405060708

# Decrypt
openssl enc -bf-cbc -d -in cipher_bfcbc.bin -out decrypted_bfcbc.txt \
  -K 00112233445566778899aabbccddeeff \
  -iv 0102030405060708
```

---

### 5. Verify Output

```bash
cat decrypted_aes128cbc.txt
cat decrypted_aes192cfb.txt
cat decrypted_bfcbc.txt
```

> All should match the original `plain.txt`.

---

### 6. Tips & Exploration

* List all supported ciphers:

  ```bash
  openssl enc -ciphers
  ```
* Try other algorithms: `des-ede3-cbc`, `aes-256-ctr`, `chacha20`.
* Keep **keys & IVs safe** – anyone with both can decrypt your data.

---

### Quick Reference

| Cipher       | Key Size | IV Size | Example `-K` (hex)                                 | Example `-iv` (hex)                |
| ------------ | -------- | ------- | -------------------------------------------------- | ---------------------------------- |
| AES-128-CBC  | 16 B     | 16 B    | `00112233445566778899aabbccddeeff`                 | `0102030405060708090a0b0c0d0e0f10` |
| AES-192-CFB  | 24 B     | 16 B    | `00112233445566778899aabbccddeeff0011223344556677` | `1112131415161718191a1b1c1d1e1f20` |
| Blowfish-CBC | 16 B     | 8 B     | `00112233445566778899aabbccddeeff`                 | `0102030405060708`                 |

---

# **BMP Encryption**

Here’s a **compact guide + code** for **Task 2: ECB vs CBC encryption of a BMP image** with OpenSSL.
(Replace filenames/keys with your own as needed.)

---

### Prepare the Image & Keys

```bash
# Original picture
cp original.bmp pic.bmp           # keep a backup

# Generate 128-bit key & IV (in hex)
openssl rand -hex 16   # key
openssl rand -hex 16   # iv
```

---

### Encrypt the BMP (ECB & CBC)

```bash
# --- ECB mode ---
openssl enc -aes-128-ecb -e -in pic.bmp -out pic_ecb_enc.bin \
  -K 00112233445566778899aabbccddeeff -nopad

# --- CBC mode ---
openssl enc -aes-128-cbc -e -in pic.bmp -out pic_cbc_enc.bin \
  -K 00112233445566778899aabbccddeeff \
  -iv 0102030405060708090a0b0c0d0e0f10 -nopad
```

 **What each part does (short):**

* `-aes-128-ecb / -aes-128-cbc` → algorithm + mode
* `-e` → encrypt
* `-in / -out` → input & output files
* `-K` → key (hex)
* `-iv` → IV for CBC (not used in ECB)
* `-nopad` → avoid padding (BMP size is already block-aligned)

---

### Fix the BMP Header (first 54 bytes for BMP)

> The first 54 bytes are the header; keep it from the original so viewers can open the encrypted data.

```bash
# Extract header from original
dd if=pic.bmp of=header.bin bs=1 count=54

# Append encrypted body
dd if=header.bin of=pic_ecb.bmp bs=1 count=54
dd if=pic_ecb_enc.bin of=pic_ecb.bmp bs=1 seek=54

dd if=header.bin of=pic_cbc.bmp bs=1 count=54
dd if=pic_cbc_enc.bin of=pic_cbc.bmp bs=1 seek=54
```

 **Explanations:**

* `dd` → copy raw bytes.
* `bs=1` → byte-wise copy.
* `count=54` → only the header.
* `seek=54` → start writing after the header.

(You can also copy the header in a hex editor like `ghex`.)

---

### View the Results

Open `pic_ecb.bmp` and `pic_cbc.bmp` in any image viewer.

* **ECB** → block patterns visible (structure leaks).
* **CBC** → image content hidden (looks random noise).

---

# **Corruption in Different Modes**

In [ ]:
#Run this first in the terminal
""" Make a 64-byte text file
echo "ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890abcdefghijklmnopqrstuvwxyz!!" > plain.txt

# Key & IV (128-bit each, hex)
KEY="00112233445566778899aabbccddeeff"
IV="0102030405060708090a0b0c0d0e0f10"

# Encrypt in different modes
openssl enc -aes-128-ecb -e -in plain.txt -out ecb.bin -K $KEY -nopad
openssl enc -aes-128-cbc -e -in plain.txt -out cbc.bin -K $KEY -iv $IV -nopad
openssl enc -aes-128-cfb -e -in plain.txt -out cfb.bin -K $KEY -iv $IV
openssl enc -aes-128-ofb -e -in plain.txt -out ofb.bin -K $KEY -iv $IV"""


----


import subprocess, shutil, os

key = "00112233445566778899aabbccddeeff"
iv  = "0102030405060708090a0b0c0d0e0f10"
modes = {
    "ECB": "-aes-128-ecb",
    "CBC": "-aes-128-cbc",
    "CFB": "-aes-128-cfb",
    "OFB": "-aes-128-ofb"
}

def corrupt(fname, byte_index=29):
    with open(fname, "rb+") as f:
        data = bytearray(f.read())
        data[byte_index] ^= 0x01
        f.seek(0); f.write(data)

for name, opt in modes.items():
    enc = f"{name.lower()}.bin"
    dec = f"{name.lower()}_dec.txt"

    # fresh copy before corruption
    shutil.copyfile(enc, f"{name.lower()}_corrupt.bin")
    corrupt(f"{name.lower()}_corrupt.bin")

    # decrypt
    cmd = ["openssl","enc",opt,"-d",
           "-in",f"{name.lower()}_corrupt.bin","-out",dec,
           "-K",key]
    if name != "ECB": cmd += ["-iv",iv]
    subprocess.run(cmd, check=True)

    print(f"\n{name} decrypted output (after 1-bit error):")
    print(open(dec).read())
    print("-"*50)

| Mode    | Effect of a 1-bit error in ciphertext                                                        |
| ------- | -------------------------------------------------------------------------------------------- |
| **ECB** | Only the block containing that bit is garbled; all other blocks decrypt perfectly.           |
| **CBC** | Corrupted block is totally wrong **and** the same bit flips in the next block. Rest is fine. |
| **CFB** | Bit flip corrupts exactly one byte of plaintext at the same position.                        |
| **OFB** | Only the bit at that position is flipped; all other bytes recover.                           |


# **Digital Signature**

<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm rsa -quiet -out rsa.pri && openssl pkey -in rsa.pri -pubout -out rsa.pub
hashi@ub-ch1:~$ openssl ecparam -name secp384r1 -genkey -out ecdsa.pri && openssl pkey -in ecdsa.pri -pubout -out ecdsa.pub
hashi@ub-ch1:~$ ls -l
total 16
-rw------- 1 hashi hashi  359 May  4 18:32 ecdsa.pri
-rw-rw-r-- 1 hashi hashi  215 May  4 18:32 ecdsa.pub
-rw------- 1 hashi hashi 1704 May  4 18:32 rsa.pri
-rw-rw-r-- 1 hashi hashi  451 May  4 18:32 rsa.pub
</pre>

#### - Here's the file I'll be signing.
<pre>
hashi@ub-ch1:~$ echo "Earth is the third planet of our Solar System." > earth.txt
</pre>

<BR>

### SIGNING AND VERIFYING USING RSAUTL
[!NOTE]
> 'openssl rsautl' is deprecated from version 3.x onwards. Please use 'openssl pkeyutl' instead.

#### - Signing earth.txt using rsautl (OLD DEPRECATED METHOD).
<pre>
hashi@ub-ch1:~$ openssl rsautl -sign -inkey rsa.pri -in earth.txt -out earth.txt.sig
The command rsautl was deprecated in version 3.0. Use 'pkeyutl' instead.
</pre>

#### - Verifying Signature using rsautl (OLD DEPRECATED METHOD).
<pre>
hashi@ub-ch1:~$ openssl rsautl -verify -inkey rsa.pub -pubin -in earth.txt.sig
The command rsautl was deprecated in version 3.0. Use 'pkeyutl' instead.
Earth is the third planet of our Solar System.
</pre>

<BR>

### GENERATING HASH BASED SIGNATURES.

#### - Generating sha1WithRSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl sha1 -sign rsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl sha1 -verify rsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

#### - Generating sha256WithRSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl sha256 -sign rsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl sha256 -verify rsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

#### - Generating sha1withECDSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl sha1 -sign ecdsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl sha1 -verify ecdsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

#### - Generating sha256WithECDSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl sha256 -sign ecdsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl sha256 -verify ecdsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

<br>

### HASH BASED DIGITAL SIGNING USING **openssl dgst** COMMAND.

**SYNTAX : ** openssl pkeyutl <options> <file_to_sign>
OPTIONS
| OPTION | DESCRIPTION |
| --- | --- |
| -sha1 | use sha1 hash. |
| -sha256 | use sha256 hash. |
| -sign <private_key> | sign using the private key from a file. |
| -verify <public_key> | verify using public key from a file. |
| -out | name of the file to write signature into. |
| -signature | file containing the signature to verify. |
| -sigopt | signing options to use. For example, pss padding. |


#### - Generating sha256withRSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl dgst -sha256 -sign rsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl dgst -sha256 -verify rsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

#### - Generating sha256WithRSA-PSS and verifying it.
<pre>
hashi@ub-ch1:~$ openssl dgst -sha256 -sign rsa.pri -sigopt rsa_padding_mode:pss -out earth.txt.sig earth.txt
hashi@ub-ch1:~$ openssl dgst -sha256 -verify rsa.pub -sigopt rsa_padding_mode:pss -signature earth.txt.sig earth.txt
Verified OK
</pre>

#### - Generating sha256WithECDSA signature and verifying it.
<pre>
hashi@ub-ch1:~$ openssl dgst -sha256 -sign ecdsa.pri -out earth.txt.sig earth.txt

hashi@ub-ch1:~$ openssl dgst -sha256 -verify ecdsa.pub -signature earth.txt.sig earth.txt
Verified OK
</pre>

<br>

### DIGITAL SIGNING USING **'openssl pkeyutl'**

**SYNTAX : ** openssl pkeyutl <options>
OPTIONS
| OPTION | DESCRIPTION |
| --- | --- |
| -sign | initiate signing operation. |
| -verify | initiate signature verification. |
| -in | name of the file to sign. |
| -out | name of the file write signatures into. |
| -sigfile | name of the file to read signature from. |
| -rawin | indicates that the data is not hashed. |
| -digest | digest algorithm to use. |
| -pkeyopt | additional public key options. E.g. PSS |


#### - Signing and verifying using sha256WithRSA.
<pre>
hashi@ub-ch1:~$ openssl pkeyutl -sign -rawin -digest sha256 -inkey rsa.pri -in earth.txt -out earth.txt.sig

hashi@ub-ch1:~$ openssl pkeyutl -verify -rawin -digest sha256 -inkey rsa.pub -pubin -sigfile earth.txt.sig -in earth.txt
Signature Verified Successfully
</pre>

#### - Signing and verifying using sha256WithECDSA.
<pre>
hashi@ub-ch1:~$ openssl pkeyutl -sign -digest sha256 -rawin -inkey ecdsa.pri -in earth.txt -out earth.txt.sig

hashi@ub-ch1:~$ openssl pkeyutl -verify -digest sha256 -rawin -inkey ecdsa.pub -pubin -sigfile earth.txt.sig -in earth.txt
Signature Verified Successfully
</pre>

#### - Signing a pre-hashed data.
<pre>
hashi@ub-ch1:~$ openssl sha256 -binary -out earth.txt.sha256 earth.txt

hashi@ub-ch1:~$ openssl pkeyutl -sign -inkey rsa.pri -in earth.txt.sha256 -out earth.txt.sig

hashi@ub-ch1:~$ openssl pkeyutl -verify -inkey rsa.pub -pubin -sigfile earth.txt.sig -in earth.txt.sha256
Signature Verified Successfully
</pre>

#### - Signing a pre-hashed data using RSA-PSS padding scheme.
<pre>
hashi@ub-ch1:~$ openssl sha256 -binary -out earth.txt.sha256 earth.txt

hashi@ub-ch1:~$ openssl pkeyutl -sign -inkey rsa.pri -pkeyopt rsa_padding_mode:pss -pkeyopt digest:sha256 -in earth.txt.sha256 -out earth.txt.sig

hashi@ub-ch1:~$ openssl pkeyutl -verify -inkey rsa.pub -pubin -pkeyopt rsa_padding_mode:pss -pkeyopt digest:sha256 -in earth.txt.sha256 -sigfile earth.txt.sig
Signature Verified Successfully
</pre>

<br>

# **Digital Certificate**

**SYNTAX:** openssl req <options>

| OPTION | DESCRIPTION |
| --- | --- |
| -x509 | Output a certificate. |
| -key | private key to use. |
| -sha256 | use sha256 hash. |
| -days | no. of days the certificate will be valid for. |
| -out file-name> | output to a file. |
| -newkey | generate a private key. |
| -noenc | don't encrypt private key. |
| -subj subject-info | Supplies subject information via the command line, bypassing interactive prompts. |

### Generating Self-Signed certificates.

*This section covers how to generate a self-signed certificate. A self-signed certificate is one in which the owner of the private key certifies their own identity.*

#### - Generate a private key and a self-signed certificate.
<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm rsa -quiet -out rsa.pri

hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer
Country Name (2 letter code) [AU]:CA
State or Province Name (full name) [Some-State]:BC
Locality Name (eg, city) []:.
Organization Name (eg, company) [Internet Widgits Pty Ltd]:Cyber Hashira
Organizational Unit Name (eg, section) []:.
Common Name (e.g. server FQDN or YOUR name) []:RSA-TEST
Email Address []:
</pre>

#### - Inspecting a certificate.
<pre>
hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -text
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            15:7f:65:5e:70:7b:f4:10:99:c8:bc:ac:14:c7:03:26:a6:bf:1d:b2
        Signature Algorithm: sha256WithRSAEncryption
</pre>

#### - Generating an RSA private key and certificate using one command.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -newkey rsa:2048 -sha256 -nodes -days 365 -out rsa.cer
Country Name (2 letter code) [AU]:CA
State or Province Name (full name) [Some-State]:BC
Locality Name (eg, city) []:.
Organization Name (eg, company) [Internet Widgits Pty Ltd]:Cyber Hashira
Organizational Unit Name (eg, section) []:
Common Name (e.g. server FQDN or YOUR name) []:RSATEST
Email Address []:

hashi@ub-ch1:~$ ls -l
total 12
-rw------- 1 hashi hashi 1704 May  7 17:41 privkey.pem
-rw-rw-r-- 1 hashi hashi 1245 May  7 17:41 rsa.cer
</pre>


#### - Generating an ECDSA private key and a self-signed certificate.
<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm ec -pkeyopt ec_paramgen_curve:secp384r1 -out ecdsa.pri

hashi@ub-ch1:~$ openssl req -x509 -key ecdsa.pri -days 365 -sha256 -subj '/CN=ECDSATEST' -out ecdsa.cer

hashi@ub-ch1:~$ openssl x509 -in ecdsa.cer -noout -text
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            52:08:a4:98:74:71:e4:5e:3c:e9:32:5f:00:da:92:df:f1:4f:09:84
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=ECDSATEST
</pre>

#### - Generating MLDSA private key and a self-signed certificate.
<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm mldsa65 -quiet -out mldsa.key

hashi@ub-ch1:~$ openssl req -x509 -key mldsa.key -days 365 -sha256 -subj '/CN=MLDSATEST' -out mldsa.cer
hashi@ub-ch1:~$ openssl x509 -in mldsa.cer -noout -text
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            57:70:91:43:ec:1f:47:1f:0e:4c:d4:2a:88:f7:7f:67:f8:36:54:ed
        Signature Algorithm: ML-DSA-65
        Issuer: CN=MLDSATEST
</pre>

<br>

### ADDING SUBJECT INFORMATION IN COMMAND LINE.
*The 'openssl req' command prompts the user to enter subject information interactively if it is not provided via the command line. This can be problematic in automated scripts. The -subj option allows subject details to be specified directly via the command line.*

#### - Adding COMMON NAME (CN) to the Subject DN.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -days 365 -out rsa.cer -subj '/CN=RSATest'

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -subject
subject=CN=RSATest
</pre>

#### - Adding CN, O, OU, C, ST, L and emailAddress to the SubjectDN.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -days 365 -sha256 -out rsa.cer -subj '/CN=RSATEST/O=CyberHashira/OU=CyberSec/C=CA/ST=BC/L=YVR/emailAddress=pki@acme.com/'

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -subject
subject=CN=RSATEST, O=CyberHashira, OU=CyberSec, C=CA, ST=BC, L=YVR, emailAddress=pki@acme.com
</pre>

<BR>

### ADDING CERTIFICATE EXTENSIONS.
*Certificate extensions are useful for defining the purpose of a certificate and restricting it to specific use cases.*

#### - Adding BasicConstraint (NON CA Certificate).
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST/O=CyberHashira' -addext "basicConstraints=CA:false"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext basicConstraints
X509v3 Basic Constraints:
    CA:FALSE
</pre>

#### - Adding BasicConstraint marked as CRITICAL with a pathlen.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST/O=CyberHashira' -addext "basicConstraints=critical, CA:true, pathlen:1"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext basicConstraints
X509v3 Basic Constraints: critical
    CA:TRUE, pathlen:1
</pre>

#### - Adding Subject Alternate Names.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST' -addext "subjectAltName= DNS:cyberhashira,IP:127.0.0.1"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext subjectAltName
X509v3 Subject Alternative Name:
    DNS:cyberhashira, IP Address:127.0.0.1
</pre>

#### - Adding Key Usage.

> [!NOTE]
> Accepted key usage values as per RFC-5280 are as follows:
> URL : https://www.rfc-editor.org/rfc/rfc5280

**Key Usage:**
- digitalSignature
- nonRepudiation
- cRLSign
- keyAgreement
- keyCertSign
- encipherOnly
- decipherOnly
- keyEncipherment
- dataEncipherment

<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST' -addext "keyUsage = digitalSignature"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext keyUsage
X509v3 Key Usage:
    Digital Signature
</pre>

#### - Adding multiple Key Usage.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST' -addext "keyUsage = digitalSignature,nonRepudiation,cRLSign"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext keyUsage
X509v3 Key Usage:
    Digital Signature, Non Repudiation, CRL Sign
</pre>

#### - Adding extended Key Usage.
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -sha256 -days 365 -out rsa.cer -subj '/CN=RSATEST' -addext "extendedKeyUsage = serverAuth, clientAuth"

hashi@ub-ch1:~$ openssl x509 -in rsa.cer -noout -ext extendedKeyUsage
X509v3 Extended Key Usage:
    TLS Web Server Authentication, TLS Web Client Authentication
</pre>

<br>

### USING CONFIGURATION FILE.

Instead of using a lengthy command to pass subject DN details and extensions, OpenSSL allows this information to be placed in a configuration file, which can then be used as input for generating certificates.


#### - A sample configuration file.

#### - EXAMPLE 1 | Generating certificate with just the Subject DN.

<pre>sample1.inf</pre>

```
[req]
distinguished_name = dname
prompt = no

commonName = CyberHashira
countryName = CA
stateOrProvinceName = BC
organizationName = CyberHashira
organizationalUnitName = PKI
emailAddress = pki@cyberhashira.com
```
</pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -config sample1.inf -out sample1.cer

hashi@ub-ch1:~$ openssl x509 -in sample1.cer -noout -subject
subject=CN=CyberHashira, C=CA, ST=BC, O=CyberHashira, OU=PKI, emailAddress=pki@cyberhashira.com
</pre>

#### - EXAMPLE 2 | Generating certificate with extensions
<pre>sample2.inf</pre>

```
[req]
x509_extensions = cert_ext
prompt = no

[ cert_ext ]
basicConstraints = CA:FALSE
keyUsage = digitalSignature, nonRepudiation
extendedKeyUsage = codeSigning
crlDistributionPoints=URI:http://myCA/ca.crl
subjectAltName = @sans
subjectKeyIdentifier=hash
authorityKeyIdentifier=keyid,issuer
authorityInfoAccess = OCSP;URI:http://ocsp.myCA/
authorityInfoAccess = caIssuers;URI:http://myCA/ca.cer
certificatePolicies= 1.2.4.5.6.7

[sans]
IP.1 = 127.0.0.1
DNS.1 = blog.cyberhashira.com
DNS.2 = video.cyberhashira.com
```
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -config sample2.inf -subj "/CN=SAMPLE2" -out sample2.cer

hashi@ub-ch1:~$ openssl x509 -in sample2.cer -noout -ext keyUsage
X509v3 Key Usage:
    Digital Signature, Non Repudiation

hashi@ub-ch1:~$ openssl x509 -in sample2.cer -noout -ext extendedKeyUsage
X509v3 Extended Key Usage:
    Code Signing

hashi@ub-ch1:~$ openssl x509 -in sample2.cer -noout -ext subjectAltName
X509v3 Subject Alternative Name:
    IP Address:127.0.0.1, DNS:blog.cyberhashira.com, DNS:video.cyberhashira.com
</pre>

#### - EXAMPLE 3 | You can also selectively specify which section of the configuration file to use.
<pre>sample3.inf</pre>

```
[req]
distinguished_name = dname
x509_extensions = cert_ext
prompt = no

[with_dname]
distinguished_name = dname
prompt = no

[with_ext]
x509_extensions = cert_ext
prompt = no

[dname]
commonName = CyberHashira
countryName = CA
stateOrProvinceName = BC
organizationName = CyberHashira
organizationalUnitName = PKI
emailAddress = pki@cyberhashira.com

[cert_ext]
basicConstraints = CA:FALSE
keyUsage = digitalSignature, nonRepudiation
extendedKeyUsage = codeSigning
crlDistributionPoints=URI:http://myCA/ca.crl
subjectAltName = @sans
subjectKeyIdentifier=hash
authorityKeyIdentifier=keyid,issuer
authorityInfoAccess = OCSP;URI:http://ocsp.myCA/
authorityInfoAccess = caIssuers;URI:http://myCA/ca.cer
certificatePolicies= 1.2.4.5.6.7

[sans]
IP.1 = 127.0.0.1
DNS.1 = blog.cyberhashira.com
DNS.2 = video.cyberhashira.com
```
<pre>
hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -config sample.inf -section with_dname -out sample1.cer

hashi@ub-ch1:~$ openssl x509 -in sample1.cer -noout -subject
subject=CN=CyberHashira, C=CA, ST=BC, O=CyberHashira, OU=PKI, emailAddress=pki@cyberhashira.com

hashi@ub-ch1:~$ openssl req -x509 -key rsa.pri -config sample.inf -section with_ext -subj "/CN=Sample3" -out sample3.cer

ashi@ub-ch1:~$ openssl x509 -in sample3.cer -noout -ext keyUsage
X509v3 Key Usage:
    Digital Signature, Non Repudiation
hashi@ub-ch1:~$ openssl x509 -in sample3.cer -noout -ext extendedKeyUsage
X509v3 Extended Key Usage:
    Code Signing
</pre>

<br>

### SIGNING A CERTIFICATE REQUEST.
Another way to generate a certificate is by signing a certificate request file. This process requires a certificate authority (CA) to sign the request. In this section, I will demonstrate how to create a CA certificate and use it to sign a certificate request.

#### - Generating CA key and certificate.

<pre>CA.inf</pre>

```
[req]
distinguished_name = dname
x509_extensions = cert_ext
prompt = no

[ dname ]
CN = RootCA

[ cert_ext ]
basicConstraints = CA:TRUE, pathlen:0
keyUsage = keyCertSign, cRLSign
subjectKeyIdentifier=hash
```
<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm rsa -quiet -out CA.pri
hashi@ub-ch1:~$ openssl req -x509 -key CA.pri -config CA.inf -out CA.cer
</pre>

#### - Generating a private key and certificate signing request.
<pre>
hashi@ub-ch1:~$ openssl genpkey -algorithm rsa -quiet -out webserver.pri

hashi@ub-ch1:~$ openssl req -new -key webserver.pri -subj '/CN=WebServer' -addext keyUsage=digitalSignature,keyEncipherment -addext extendedKeyUsage=serverAuth -out webserver.csr
</pre>

#### - Here's how to read certificate signing request file.
<pre>
hashi@ub-ch1:~$ openssl req -in webserver.csr -noout -text
Certificate Request:
    Data:
        Version: 1 (0x0)
        Subject: CN=WebServer
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (2048 bit)
</pre>

#### - Signing certificate request.
<pre>
hashi@ub-ch1:~$ openssl x509 -req -days 365 -in webserver.csr -CA CA.cer -CAkey CA.pri -CAcreateserial -out webserver.cer
Certificate request self-signature ok
subject=CN=WebServer

hashi@ub-ch1:~$ openssl x509 -in webserver.cer -noout -text
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            24:53:82:9a:99:3f:87:5f:11:55:1c:56:c1:76:9c:67:bf:0d:76:c0
        Signature Algorithm: sha256WithRSAEncryption
        Issuer: CN=RootCA
        Validity
            Not Before: May  9 01:23:37 2025 GMT
            Not After : May  9 01:23:37 2026 GMT
        Subject: CN=WebServer
</pre>

<br>

# **HTTPS using a self-signed certificate**

### Using Python’s Built-in HTTP Server with SSL

If you have Python installed, it’s super easy to set up an HTTPS server with SSL.

#### Steps:

1. **Ensure Python is installed** (Python 3.x is recommended):

   * Check by running `python --version` or `python3 --version` in your terminal.

2. **Generate a self-signed certificate** (same steps as before):
   Run the following commands in your terminal (from the directory where your `hello.html` file is located):

   ```bash
   openssl genpkey -algorithm RSA -out localhost.key -aes256
   openssl req -new -key localhost.key -out localhost.csr
   openssl x509 -req -in localhost.csr -signkey localhost.key -out localhost.crt
   ```

3. **Start the HTTPS server**:
   Use Python to serve your HTML file over HTTPS. Run this command in the terminal (from the directory containing your HTML file):

   ```bash
   python3 -m http.server 8000 --bind 127.0.0.1 --ssl-key localhost.key --ssl-cert localhost.crt
   ```

   This will start a simple HTTPS server on `https://localhost:8000`.

4. **Access your site**:
   Open your browser and visit `https://localhost:8000`. You’ll see your HTML file served over HTTPS. Since it's self-signed, you may need to accept the security warning in the browser.

In [ ]:
"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hello World</title>
</head>
<body>
    <h1>Hello, World!</h1>
    <p>This is a simple HTML website for testing purposes.</p>
</body>
</html>"""

In [ ]:
import http.server
import ssl

# Set the handler to serve files in the current directory
handler = http.server.SimpleHTTPRequestHandler

# Create the HTTP server on localhost, port 8000
httpd = http.server.HTTPServer(('127.0.0.1', 6210), handler)

# Create an SSL context and wrap the server socket
context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
context.load_cert_chain(certfile="localhost.crt", keyfile="localhost.key", password="password")

# Apply SSL to the HTTP server's socket
httpd.socket = context.wrap_socket(httpd.socket, server_side=True)

print("Server started at https://localhost:8080")

# Start the server
httpd.serve_forever()